In [1]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from scipy import ndimage, misc
import tensorflow as tf
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from os import listdir
from os.path import isfile, join
import gc
from astropy.io import fits
import matplotlib.pyplot as plt
import random
from numpy.random import default_rng
import numpy as np
from tensorflow.keras import datasets, layers, models
rng = default_rng(42)
import glob
import seaborn as sns
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from mlxtend.plotting import plot_confusion_matrix

2024-05-29 11:22:40.986619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

Bad key text.latex.preview in file /opt/anaconda/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.7.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /opt/anaconda/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
ht

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
carpetas = ['fits_inclinated','zvar','z_cero']
path = '/data/processing/avera/'

In [7]:
i=1
import h5py
with h5py.File(path + carpetas[i]+"_dataset.h5", 'r') as hdf:
    # Leer set de entrenamiento
    x_train = hdf['x_train'][:]
    train_labels = hdf['train_labels'][:]
    
    # Leer set de validación
    x_val = hdf['x_val'][:]
    val_labels = hdf['val_labels'][:]
    
    # Leer set de testeo
    x_test = hdf['x_test'][:]
    test_labels = hdf['test_labels'][:]

In [8]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_gen = train_datagen.flow(
    x_train,
    train_labels,
    batch_size=64,shuffle=True
)

val_gen = validation_datagen.flow(
    x_val,
    val_labels,
    batch_size=32,shuffle=True
)

In [6]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(300, 300, 3),
    include_top=False)
base_model.trainable = False
inputs = keras.Input(shape=(300, 300, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1,activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.load_weights(f'Checkpoints/{carpetas[i]}/checkpoints/my_checkpoint')


2024-05-29 11:23:54.047182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 11:23:54.055035: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy()])

history = model.fit(train_gen, epochs=100, 
                    validation_data=val_gen)

Epoch 1/100
110/110 [==============================] - 339s 3s/step - loss: 0.4207 - binary_accuracy: 0.8165 - val_loss: 0.3286 - val_binary_accuracy: 0.8600
Epoch 2/100
110/110 [==============================] - 295s 3s/step - loss: 0.3993 - binary_accuracy: 0.8269 - val_loss: 0.3215 - val_binary_accuracy: 0.8677
Epoch 3/100
110/110 [==============================] - 297s 3s/step - loss: 0.3828 - binary_accuracy: 0.8348 - val_loss: 0.3148 - val_binary_accuracy: 0.8666
Epoch 4/100
110/110 [==============================] - 298s 3s/step - loss: 0.3695 - binary_accuracy: 0.8429 - val_loss: 0.3078 - val_binary_accuracy: 0.8688
Epoch 5/100
110/110 [==============================] - 298s 3s/step - loss: 0.3591 - binary_accuracy: 0.8466 - val_loss: 0.3021 - val_binary_accuracy: 0.8754
Epoch 6/100
110/110 [==============================] - 296s 3s/step - loss: 0.3500 - binary_accuracy: 0.8487 - val_loss: 0.2960 - val_binary_accuracy: 0.8787
Epoch 7/100
110/110 [==============================]

Exception ignored in: <function UniquePtr.__del__ at 0x7f41f5d0e280>
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.8/site-packages/tensorflow/python/framework/c_api_util.py", line 70, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 